In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


In [26]:
train_data = pd.read_csv('data/msr_paraphrase_corpus/msr_paraphrase_train.txt', sep = "\t", header = None, names = ['class', 'id1', 'id2', 'text1', 'text2']) 
train_data = train_data.drop([0])
train_data['text'] = train_data['text1']+' '+train_data['text2']
train_data.drop(['id1', 'id2'], axis = 1, inplace = True) 
train_data = train_data.dropna()
train_data['class'] = train_data['class'].apply(lambda x: int(x))


In [30]:
def tok_helper(word):
    word = word.replace(".","").replace(",","")
    return word

def lemma_tokenizer(text):
    wpt = WordPunctTokenizer()
    lemmatizer=WordNetLemmatizer()
    return [lemmatizer.lemmatize(tok_helper(w)) for w in wpt.tokenize(text) if w not in stopwords.words('english')]

vectorizer = CountVectorizer(tokenizer=lemma_tokenizer)
text_vector = vectorizer.fit_transform(train_data['text']) 

classifier = LogisticRegression(max_iter=1000)
classifier.fit(text_vector, train_data['class'].values)
# classifier.fit

LogisticRegression(max_iter=1000)

In [27]:
# test
test_data = pd.read_csv('data/msr_paraphrase_corpus/msr_paraphrase_test.txt', sep = "\t", header = None, names = ['class', 'id1', 'id2', 'text1', 'text2']) 
test_data = test_data.drop([0])
test_data['text'] = test_data['text1']+' '+test_data['text2']
test_data.drop(['id1', 'id2'], axis = 1, inplace = True) 
test_data = test_data.dropna()
test_data['class'] = test_data['class'].apply(lambda x: int(x))
